In [ ]:
import json
from collections import Counter
import pickle
import numpy as np
import pandas as pd
import os
import sys
import argparse
from os.path import dirname, realpath
import math
import hashlib
import datetime
import datetime as datetime
sys.path.append("./src/")

import torch
from torch import nn, optim
from torch.nn import functional as F

from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
#import plotter
import matplotlib.pyplot as plt
sys.path.append(dirname(dirname(realpath(os.getcwd()))))

In [ ]:
src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))
pat_ids = (testDF['patient_id'] ).astype(int)
testDF['pids'] = pat_ids.apply(md5)

In [ ]:
def compute_calibration_curve(golds_for_eval,Probs_for_eval):
    prob_true, prob_pred = calibration_curve(golds_for_eval, Probs_for_eval, n_bins=40)
    return prob_true, prob_pred


In [ ]:
def get_4Ms(M):
    M1 = M.drop_duplicates(['patient_id'])
    m0 = M.duplicated(['patient_id'])
    M234 = M[m0]
    M2 = M234.drop_duplicates(['patient_id'])

    m00 = M234.duplicated(['patient_id'])
    M34 = M234[m00]
    M3 = M34.drop_duplicates(['patient_id'])

    m000 = M34.duplicated(['patient_id'])
    M4 = M34[m000]
    return M1, M2, M3, M4

In [ ]:
tdf = pd.DataFrame({'patient_id': (testDF['patient_id'] ),    
                    'pids': (testDF['pids'] ),    
                  'dob': (testDF['dob'] ),  
                  'outcome_date': (testDF['outcome_date'] ), 
                  'obs_time_end': (testDF['obs_time_end'] ),  
                  'index_date': (testDF['index_date'] ),  
                  'diag_date': (testDF['diag_date'] ),  
                  'outcome': (testDF['outcome'])    })


In [ ]:
devpath = os.path.join(src_path ,'logs_transformer_vte/K/12_4/', "a8e44cbb2213334954e85de32bdcaf2c.results.dev_preds")
#devpath = os.path.join(src_path ,'logs_transformer_vte/K/12_4/', "6a827773e837f2b6ddb9b70a2821c81f.results.dev_preds")
with open(devpath, 'rb') as f:
    R = pickle.load(f)
    p = np.array(R['probs'])
Df = pd.DataFrame.from_dict(R)
Df['probs'] = Df['probs'].astype(float)
Df['exams'] = Df['exams'].astype(int)
M = pd.merge(tdf, Df)
M1, M2, M3, M4 = get_4Ms(M)

In [ ]:
testpath = os.path.join(src_path ,'logs_transformer_vte/K/12_4/', "a8e44cbb2213334954e85de32bdcaf2c.results.test_preds")
#testpath = os.path.join(src_path ,'logs_transformer_vte/K/12_4/', "6a827773e837f2b6ddb9b70a2821c81f.results.test_preds")
with open(testpath, 'rb') as f:
    R_t = pickle.load(f)
Df_test = pd.DataFrame.from_dict(R_t)
Df_test['probs'] = Df_test['probs'].astype(float)
M_test = pd.merge(tdf, Df_test)
M_test1, M_test2, M_test3, M_test4 = get_4Ms(M_test)

In [ ]:
def logit (p): return np.log(  p / (1 - p) )
def expit (p): return  1/(1 + np.exp(-p))

In [ ]:
mp0 = np.array(M['probs'])
mp1 = np.array(M1['probs'])
mp2 = np.array(M2['probs'])
mp3 = np.array(M3['probs'])
mp4 = np.array(M4['probs'])

In [ ]:
tp0 = np.array(M_test['probs'])
tp1 = np.array(M_test1['probs'])
tp2 = np.array(M_test2['probs'])
tp3 = np.array(M_test3['probs'])
tp4 = np.array(M_test4['probs'])

In [ ]:
gold0 =   np.array(M['golds'],dtype = int)
gold1 =   np.array(M1['golds'],dtype = int)
gold2 =   np.array(M2['golds'],dtype = int)
gold3 =   np.array(M3['golds'],dtype = int)
gold4 =   np.array(M4['golds'],dtype = int)

In [ ]:
tgold1 =   np.array(M_test1['golds'],dtype = int)
tgold2 =   np.array(M_test2['golds'],dtype = int)
tgold3 =   np.array(M_test3['golds'],dtype = int)
tgold4 =   np.array(M_test4['golds'],dtype = int)
tgold =   np.array(M_test['golds'],dtype = int)

In [ ]:
def platt (y_prob, y_true, p_test):
    logistic = LogisticRegression(C=1e4, solver='lbfgs')
    y_logit =  (y_prob) ###
    logistic.fit(y_logit.reshape(-1,1), y_true)
    coef_ = logistic.coef_[0]
    intercept_ = logistic.intercept_
    fit =  (p_test) * coef_[0] + intercept_
    out = 1 / (1 + np.exp(-fit))
    return  out

In [ ]:
global_cal_dev =platt (np.array(M['probs']), np.array(M ['golds'],dtype = int),  M ['probs']  )
#global_cal_dev =platt (np.array(M['probs']), np.array(M ['golds'],dtype = int),  M_test ['probs']  )

In [ ]:
plt.hist( global_cal_dev )

In [ ]:
M ['cal'] = global_cal_dev

In [ ]:
D_cal1, D_cal2, D_cal3, D_cal4 = get_4Ms(M)
cp1 = np.array(D_cal1['cal'])
cp2 = np.array(D_cal2['cal'])
cp3 = np.array(D_cal3['cal'])
cp4 = np.array(D_cal4['cal'])

In [ ]:
plt.hist(cp1)

In [ ]:
f1 =platt (cp1,  np.array(M1['golds'],dtype = int), tp1 )
f2 =platt (cp2,  np.array(M2['golds'],dtype = int),  tp2 )
f3 =platt (cp3,  np.array(M3['golds'],dtype = int),  tp3 )
f4 =platt (cp4,  np.array(M4['golds'],dtype = int), tp4  )

In [ ]:
y1, p1 = compute_calibration_curve(M_test1['golds'], f1 )
y2, p2 = compute_calibration_curve(M_test2['golds'], f2 )
y3, p3 = compute_calibration_curve(M_test3['golds'], f3 )
y4, p4 = compute_calibration_curve(M_test4['golds'], f4 )


In [ ]:
import matplotlib.pyplot as plt

def plotter (p_i, y_i):
    plt.figure(figsize=(5, 5))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))
    ax1.plot(p_i, y_i, 's-')
    ax1.plot([0, 1], [0, 1], linestyle='--', 
             color='grey', label='random guess (AUROC: 0.500)')
    ax2.hist(p_i, bins=10, range=(0, 1))
    ax1.set_xlim([-0.01, 1.01])
    ax1.set_ylim([-0.01, 1.01])
    ax1.set_xlabel('Predicted value')
    ax1.set_ylabel('Fraction of positive')
    ax1.set_title("Calibration plot")
    ax2.set_xlabel('Prediction probability')
    ax2.set_ylabel('Count')
    plt.tight_layout()
    

In [ ]:
y00, p00 = compute_calibration_curve(M_test['golds'], tp0 )

plotter(p00, y00)

In [ ]:
plotter(p1, y1)

In [ ]:
plotter(p2, y2)

In [ ]:
plotter(p3, y3)

In [ ]:
plotter(p4, y4)

In [ ]:
# how many people had event within 6 months of index date

In [ ]:
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score

In [ ]:
tgold


In [ ]:
roc_auc_score(tgold, tp0, average = 'samples')


In [ ]:
roc_auc_score(tgold1, tp1, average = 'samples')


In [ ]:
roc_auc_score(tgold2, tp2, average = 'samples')


In [ ]:
roc_auc_score(tgold3, tp3, average = 'samples')


In [ ]:
roc_auc_score(tgold4, tp4, average = 'samples')


In [ ]:
np.mean(tgold)**.5

In [ ]:
( confusion_matrix (tgold  , cal0>.05) )

In [ ]:
( confusion_matrix (tgold4  , cal4>.5) )

In [ ]:
( confusion_matrix (tgold3 , cal3>.5) )

In [ ]:
( confusion_matrix (tgold2 , cal2>.5) )

In [ ]:
( confusion_matrix (tgold1, cal1>.5) )

In [ ]:
def get_coef (y_prob, y_true, p_test):
    logistic = LogisticRegression(C=1e4, solver='lbfgs')
    logistic.fit(y_prob.reshape(-1,1), y_true)
    coef_ = logistic.coef_[0]
    intercept_ = logistic.intercept_
    fit = p_test * coef_[0] + intercept_
    return  coef_ , intercept_

In [ ]:
m1.to_csv('output/XCoh_Sen.csv') 

In [ ]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 